In [1]:
import json

In [2]:
with open("../../data/mb_jobs_7.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [3]:
data[0]['salary_range']

{'junior': {'min': 9000000, 'max': 25000000},
 'middle': {'min': 12000000, 'max': 30000000},
 'professional': {'min': 15000000, 'max': 35000000}}

In [4]:
import re

def parse_salaries(salary_data):
    if isinstance(salary_data, str):
        salaries = ("", "")
        lines = salary_data.split('\n')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # Check for "a - b VND" format
            match = re.match(r'(\d{1,3}(?:,\d{3})*) - (\d{1,3}(?:,\d{3})*)\s*VNĐ', line)
            if match:
                min_salary = match.group(1).replace(',', '')
                max_salary = match.group(2).replace(',', '')
            else:
                # For other formats, set min and max to the same value
                # Remove "VNĐ" and any leading/trailing whitespace
                value = line.replace('VNĐ', '').strip()
                
                # Try to extract a number if present
                number_match = re.search(r'\d{1,3}(?:,\d{3})*', value)
                if number_match:
                    salary_value = int(number_match.group().replace(',', ''))
                    min_salary = max_salary = salary_value
                else:
                    min_salary = 0
                    max_salary = -1

            salaries = (min_salary, max_salary)
        
        return salaries
    else:
        return (salary_data["junior"]["min"], salary_data["professional"]["max"])
            

def is_number(s):
    return s.isdigit()

In [5]:
refined_data = []

for entry in data:    
    salary_range = parse_salaries(entry["salary_range"])
    min_salary = salary_range[0]
    max_salary = salary_range[1]
    refined_data.append({
        "url": entry["url"],
        "job_title": entry["job_title"],
        "department": entry["department"],
        "job_code": entry["job_code"],
        "workplace": entry["workplace"],
        "job_rank": entry["job_rank"],
        "job_type": entry["job_type"],
        "job_industry": [term.strip() for term in entry["job_industry"].split(",")],
        "deadline_application": entry["deadline_application"],
        "min_salary": min_salary,
        "max_salary": max_salary,
        "job_detail": entry["job_detail"],
        "job_requirements": entry["job_requirements"]
    })

In [6]:
import pandas as pd

df = pd.DataFrame(refined_data)

In [7]:
def contains_any(industry_list, search_terms):
    return any(any(term.lower() in industry.lower() for term in search_terms) for industry in industry_list)

In [8]:
search_terms = ['Kinh doanh']
filtered_df = df[df['job_industry'].apply(lambda x: contains_any(x, search_terms))]

In [9]:
def get_unique_job_industries(df, column_name='job_industry'):
    """
    Returns a sorted list of unique job industries from a DataFrame column.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing the job industry data.
    column_name (str): The name of the column containing job industry lists. Default is 'job_industry'.
    
    Returns:
    list: A sorted list of unique job industries.
    """
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the DataFrame")
    
    # Flatten the list of lists and get unique values
    unique_industries = sorted(set(industry 
                                   for industries in df[column_name] 
                                   for industry in industries))
    
    return unique_industries


# Get unique job industries
unique_industries = get_unique_job_industries(df)

print("Unique Job Industries:")
for industry in unique_industries:
    print(f"{industry}")

# You can also use it with a different column name if needed
# For example, if your column was named 'industries' instead of 'job_industry':
# unique_industries = get_unique_job_industries(df, column_name='industries')

Unique Job Industries:
Bán hàng / Kinh doanh
Bảo trì / Sửa chữa
Bộ phận pháp lý
CNTT - Phần cứng / Mạng
CNTT - Phần mềm
Dịch vụ khách hàng
Hành chính / Thư ký
Kế toán / Kiểm toán
Luật / Pháp lý
Mỹ thuật / Nghệ thuật / Thiết kế
Ngân hàng
Nhân sự
Quản lý điều hành
Quảng cáo / Đối ngoại / Truyền Thông
Tài chính / Đầu tư
Tài chính kế hoạch
Tư vấn


In [11]:
df['workplace'].unique()

array(['Khánh Hòa', 'Hà Nam', 'Đà Nẵng', 'Đắk Lắk', 'Phú Yên', 'Nam Định',
       'Đắk Nông', 'Bình Định', 'Hải Phòng', 'Hà Nội', 'Hồ Chí Minh',
       'Bình Thuận', 'Cần Thơ', 'Sóc Trăng', 'Long An', 'Cà Mau',
       'Tiền Giang', 'Tây Ninh', 'Bình Dương', 'Kiên Giang', 'Đồng Nai',
       'Lâm Đồng', 'Đồng Tháp', 'Bình Phước', 'Bến Tre', 'Trà Vinh',
       'Ninh Bình', 'Hà Tĩnh', 'Thanh Hóa', 'Nghệ An', 'Yên Bái',
       'Vĩnh Phúc', 'Phú Thọ', 'Tuyên Quang', 'Bắc Ninh', 'Thái Nguyên',
       'Sơn La', 'Lào Cai', 'Lạng Sơn', 'Hưng Yên', 'Hòa Bình',
       'Hải Dương', 'Điện Biên', 'Bắc Giang', 'An Giang', 'Bạc Liêu',
       'Hậu Giang', 'Vĩnh Long', 'Bà Rịa - Vũng Tàu', 'Thừa Thiên Huế',
       'Quảng Trị', 'Quảng Ninh', 'Thái Bình', 'Gia Lai', 'Quảng Ngãi',
       'Quảng Nam', 'Quảng Bình', 'Ninh Thuận'], dtype=object)

In [12]:
df['job_title'].unique()

array(['Chuyên viên Khách hàng Cá nhân',
       'Chuyên viên Tập sự Khách hàng Cá nhân',
       'Chuyên viên Tập sự Khách hàng Doanh nghiệp',
       'Chuyên viên Khách hàng Doanh nghiệp',
       'Chuyên gia Phát triển dịch vụ (API)',
       'Chuyên viên Phát triển dịch vụ thẻ Khách hàng doanh nghiệp (BA thẻ)',
       'Cộng tác viên Tuyển dụng', 'Chuyên viên UB',
       'Chuyên viên Vận hành hệ thống', 'Data Platform Engineer',
       'Quality Assurance (QA)', 'UI UX Designer',
       'Chuyên viên Thúc đẩy bán Sản phẩm Tín dụng', 'Giao dịch viên',
       'Kiểm soát viên', 'Kiểm toán viên (mảng Mô hình, Dữ liệu)',
       'Phó phòng Khách hàng Cá nhân', 'Trưởng phòng Khách hàng Cá nhân',
       'Chuyên viên và Chuyên viên cao cấp Quản trị rủi ro mô hình',
       'Chuyên viên và Chuyên viên cao cấp Kiểm định Mô hình rủi ro',
       'Chuyên viên cao cấp Phân tích nghiệp vụ',
       'Chuyên viên và Chuyên viên cao cấp Xây dựng Mô hình rủi ro',
       'Red Team', 'Pentester', 'Chuyên viên kiế